In [155]:
# pip install scikit-learn

In [156]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import torch
import numpy as np
from sklearn.metrics import classification_report, hamming_loss, accuracy_score
from sklearn.metrics import f1_score


In [157]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [158]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [159]:
df = pd.read_csv("/content/drive/MyDrive/chexpert_data_v2/valid.csv", sep=",")

In [160]:
image_embedding_path = "/content/drive/MyDrive/Embedings/Image_embeding/image_embeddings_LRF30%.pt"
text_embedding_path = "/content/drive/MyDrive/Embedings/Text_embeding/disease_text_embeddings.pt"

In [161]:
image_embeddings = torch.load(image_embedding_path)
text_embeddings = torch.load(text_embedding_path)

In [162]:
disease_columns = list(text_embeddings.keys())

In [163]:
disease_columns

['Atelectasis',
 'Cardiomegaly',
 'Consolidation',
 'Edema',
 'Pleural Effusion',
 'Pneumonia',
 'Pneumothorax']

In [164]:
print(df.columns)


Index(['Path', 'Sex', 'Age', 'Frontal/Lateral', 'AP/PA', 'No Finding',
       'Enlarged Cardiomediastinum', 'Cardiomegaly', 'Lung Opacity',
       'Lung Lesion', 'Edema', 'Consolidation', 'Pneumonia', 'Atelectasis',
       'Pneumothorax', 'Pleural Effusion', 'Pleural Other', 'Fracture',
       'Support Devices'],
      dtype='object')


In [165]:
disease_columns = [
    'Atelectasis',
    'Cardiomegaly',
    'Consolidation',
    'Edema',
    'Pleural Effusion',
    'Pneumonia',
    'Pneumothorax'
]




In [166]:
label_dict = {}
prefix = "CheXpert-v1.0-small/"

for i, row in df.iterrows():
    path = row["Path"]

    if path.startswith(prefix):
        path = path[len(prefix):]

    label = []
    for disease in disease_columns:
        val = row[disease]
        if pd.isna(val):
            label.append(0.0)
        else:
            label.append(float(val))
    label_dict[path] = torch.tensor(label)


In [167]:
img_paths = list(image_embeddings.keys())


In [168]:
# img_paths

In [169]:
labels = torch.stack([label_dict[p] for p in img_paths])


In [170]:
labels

tensor([[0., 1., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 1., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 1., 0., 0.]])

In [171]:
img_keys = list(image_embeddings.keys())
img_tensor = torch.stack([image_embeddings[k] for k in img_keys])  # [N, r]

disease_keys = list(text_embeddings.keys())
text_tensor = torch.stack([text_embeddings[k] for k in disease_keys])  # [C, r]


In [172]:
print(img_tensor.size())
print(text_tensor.size())


torch.Size([234, 197, 64])
torch.Size([7, 768])


In [173]:
image_repr = img_tensor.mean(dim=1)
text_tensor = torch.stack([text_embeddings[k] for k in text_embeddings]).to(device)  # [7, 768]
text_tensor = F.normalize(text_tensor, dim=1)


In [174]:
proj = torch.nn.Linear(64, 768).to(device)
image_proj = proj(image_repr)  # [234, 768]


In [175]:
image_proj = F.normalize(image_proj, dim=1)
text_tensor = F.normalize(text_tensor, dim=1)


In [176]:
similarity = image_proj @ text_tensor.T  # [234, 7]


In [177]:
print(image_repr.size())

torch.Size([234, 64])


In [178]:
optimizer = torch.optim.Adam(proj.parameters(), lr=1e-4)

In [179]:
# labels = labels.to(device)

In [180]:
probs = torch.sigmoid(similarity)  # [234, 7]
preds = (probs > 0.5).int()


In [181]:
threshold = 0.5
preds = (probs > threshold).float()


In [182]:
preds

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [1., 1., 1.,  ..., 1., 1., 1.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 1.,  ..., 1., 0., 0.]])

In [183]:
accuracy = (preds == labels).all(dim=1).float().mean()
print(f"Exact match accuracy: {accuracy.item():.4f}")


Exact match accuracy: 0.3419


In [184]:
sample_accuracy = (preds == labels).float().mean()
print(f"Sample-wise accuracy: {sample_accuracy.item():.4f}")


Sample-wise accuracy: 0.7198


In [185]:
per_label_acc = ((preds == labels).float().mean(dim=0))
for i, disease in enumerate(disease_columns):
    print(f"{disease}: {per_label_acc[i].item():.4f}")

macro_accuracy = per_label_acc.mean()
print(f"Macro (mean per-label) accuracy: {macro_accuracy.item():.4f}")


Atelectasis: 0.6282
Cardiomegaly: 0.7350
Consolidation: 0.7350
Edema: 0.7991
Pleural Effusion: 0.5983
Pneumonia: 0.7821
Pneumothorax: 0.7607
Macro (mean per-label) accuracy: 0.7198


In [186]:
y_true = labels.cpu().numpy()
y_pred = preds.cpu().numpy()

target_names = ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema',
                'Pleural Effusion', 'Pneumonia', 'Pneumothorax']

print("🔍 Classification Report (Macro & Per-label):")
print(classification_report(y_true, y_pred, target_names=target_names, zero_division=0))

#  Hamming Loss
hl = hamming_loss(y_true, y_pred)
print(f"\n Hamming Loss: {hl:.4f}")

# ⚖️ Exact Match Ratio (sample-wise accuracy)
exact_match = np.all(y_true == y_pred, axis=1).mean()
print(f" Exact Match Accuracy: {exact_match:.4f}")

🔍 Classification Report (Macro & Per-label):
                  precision    recall  f1-score   support

     Atelectasis       0.41      0.21      0.28        80
    Cardiomegaly       0.55      0.51      0.53        68
   Consolidation       0.28      0.58      0.38        33
           Edema       0.49      0.73      0.58        45
Pleural Effusion       0.29      0.27      0.28        67
       Pneumonia       0.04      0.25      0.07         8
    Pneumothorax       0.00      0.00      0.00         8

       micro avg       0.31      0.40      0.35       309
       macro avg       0.29      0.36      0.30       309
    weighted avg       0.39      0.40      0.38       309
     samples avg       0.09      0.22      0.12       309


 Hamming Loss: 0.2802
 Exact Match Accuracy: 0.3419


In [187]:
best_threshold = 0.0
best_f1 = 0.0

for t in np.arange(0.1, 0.9, 0.05):
    preds = (probs > t).int()
    f1 = f1_score(y_true, preds.numpy(), average='macro', zero_division=0)
    if f1 > best_f1:
        best_f1 = f1
        best_threshold = t

print(f"  threshold: {best_threshold:.2f} با F1: {best_f1:.4f}")


  threshold: 0.50 با F1: 0.3036
